In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

supermarket_df = pd.read_csv("完整数据.csv"
                             ,encoding="Windows-1252")
# 邮编缺失值过多，删除
supermarket_df.drop(columns="Postal Code", axis=0, inplace=True)

supermarket_df["Order Date"] = pd.to_datetime(supermarket_df["Order Date"], format="mixed")
supermarket_df["Ship Date"] = pd.to_datetime(supermarket_df["Ship Date"], format="mixed")
supermarket_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,...,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.1400,35.46,Medium
1,22253,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.0360,9.72,Medium
2,48883,HU-2011-1220,2011-01-01,2011-05-01,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,...,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.6400,8.17,High
3,11731,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,...,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.0550,4.82,High
4,22255,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.7700,4.70,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,32593,CA-2014-115427,2014-12-31,2015-04-01,Standard Class,EB-13975,Erica Bern,Corporate,Fairfield,California,...,OFF-BI-10002103,Office Supplies,Binders,"Cardinal Slant-D Ring Binder, Heavy Gauge Vinyl",13.904,2,0.2,4.5188,0.89,Medium
51286,47594,MO-2014-2560,2014-12-31,2015-05-01,Standard Class,LP-7095,Liz Preis,Consumer,Agadir,Souss-Massa-Draâ,...,OFF-WIL-10001069,Office Supplies,Binders,"Wilson Jones Hole Reinforcements, Clear",3.990,1,0.0,0.4200,0.49,Medium
51287,8857,MX-2014-110527,2014-12-31,2015-02-01,Second Class,CM-12190,Charlotte Melton,Consumer,Managua,Managua,...,OFF-LA-10004182,Office Supplies,Labels,"Hon Color Coded Labels, 5000 Label Set",26.400,3,0.0,12.3600,0.35,Medium
51288,6852,MX-2014-114783,2014-12-31,2015-06-01,Standard Class,TD-20995,Tamara Dahlen,Consumer,Juárez,Chihuahua,...,OFF-LA-10000413,Office Supplies,Labels,"Hon Legal Exhibit Labels, Alphabetical",7.120,1,0.0,0.5600,0.20,Medium


# 计算RFM值，最后时间采用2015-01-01

In [3]:
import datetime
rfm = supermarket_df[["Order Date", "Customer ID", "Sales"]].copy()
rfm = rfm.groupby("Customer ID").agg(
  R=('Order Date', lambda x: (datetime.datetime(2015, 1, 1, 0, 0, 0) - max(x)).days),
  F=('Customer ID', 'count'),
  M=('Sales', sum)
)
rfm

,R,F,M
Customer ID,,,
AA-10315,9,42,13747.41300
AA-10375,7,42,5884.19500
AA-10480,118,38,17695.58978
AA-10645,27,73,15343.89070
AA-315,3,8,2243.25600
...,...,...,...
YS-21880,10,54,18703.60600
ZC-11910,201,1,7.17300
ZC-21910,4,84,28472.81926


# 对数据进行标准化

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scalar = MinMaxScaler()
x = scalar.fit_transform(rfm)
pd.DataFrame(x, columns=rfm.columns, index=rfm.index)

,R,F,M
Customer ID,,,
AA-10315,0.007156,0.427083,0.385302
AA-10375,0.005367,0.427083,0.164803
AA-10480,0.104651,0.385417,0.496016
AA-10645,0.023256,0.750000,0.430070
AA-315,0.001789,0.072917,0.062704
...,...,...,...
YS-21880,0.008050,0.552083,0.524283
ZC-11910,0.178891,0.000000,0.000000
ZC-21910,0.002683,0.864583,0.798230


# 参数搜索

In [5]:
from sklearn.cluster import KMeans
import altair as alt
sse = pd.DataFrame(columns=["K", "SSE"])
for k in range(2, 10):
  model = KMeans(n_clusters=k)
  model.fit(x)
  sse.loc[len(sse)] = [k, model.inertia_]

alt.Chart(sse).mark_line().encode(
  x="K:O",
  y="SSE"
).properties(
  width=600
)

  File "d:\miniconda3\envs\dataanalysis\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\dataanalysis\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\envs\dataanalysis\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "d:\miniconda3\envs\dataanalysis\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


alt.Chart(...)

# 训练模型

In [34]:
model = KMeans(n_clusters=4) # best k = 4
model.fit(x)

KMeans(n_clusters=5)

# 可视化

In [36]:
a = pd.Series(model.labels_).value_counts().reset_index()
a = a.assign(index=["A","B","C","D"])

a["percentage"] = (a["count"] / a["count"].sum() * 100).apply(lambda x: f"{x:.2f}%")
print(a)
pie = alt.Chart(a).mark_arc(outerRadius=160).encode(
  theta=alt.Theta("count").stack(True),
  color=alt.Color("index:N")
)
text = pie.mark_text(radius=180, size=10).encode(
  text="percentage:N",
)
(pie + text)

  index  count percentage
0     A    565     35.53%
1     B    476     29.94%
2     C    301     18.93%
3     D    195     12.26%
4     E     53      3.33%


alt.LayerChart(...)

In [37]:
b = pd.Series(model.labels_).value_counts().reset_index()
# ABCD: 1023
# BACD
b = pd.DataFrame(model.cluster_centers_, index=["B","A","C","D",], columns=["R","F","M"]).sort_index().reset_index()
c = pd.DataFrame(columns=["Group", "Score", "Value"])
t = ""
for i in b.iterrows():
  for k in zip("GRFM", i[1]):
    if (k[0] == "G"):
      t = k[1]
      continue
    c.loc[len(c)] = [t, k[0], k[1]]
alt.Chart(c).mark_bar().encode(
  x="Score:N",
  xOffset="Group:N",
  y="Value:Q",
  color=alt.Color("Group:N")
).properties(
  width=500,
  height=300
)

alt.Chart(...)